In [4]:
from math import pi
import pandas as pd
from bokeh.io import show, push_notebook
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.models import ColumnDataSource
import json
import time
import subprocess
from bokeh.plotting import figure
import numpy as np 
from bokeh.io import output_notebook, show

output_notebook()

PROJECT_PATH = '/Users/mac/Documents/GitHub/INFO6205-FINAL'

def get_config():
    with open(f'{PROJECT_PATH}/config/config.json') as f:
        config = json.load(f)
    return config


def get_raw_data(config):
    json_output = subprocess.check_output(['python3', f'{PROJECT_PATH}/Model/script.py'], input=json.dumps(config).encode('utf8'))
    return json.loads(json_output)


def gen_data(config, raw_data):
    for n, c, tot in zip(raw_data['new_cases'], raw_data['cure_cases'], raw_data['total_cases']):
        POPULATION = config['TOTAL_PPL'] 
        x = {
            'New Cases': n,
            'Pre-existing Cases': tot - n,
            'Cured Cases': c,
            'Non-infected Cases': POPULATION - tot - c
        }

        data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'label'})
        data['angle'] = data['value'] / data['value'].sum() * 2 * pi
        data['color'] = ['#CD1919', '#CD9696', '#96EE96', '#BECCC2']
        yield data

def plot(user_config):
    p = figure(plot_height=350, title="Pie Chart", toolbar_location=None,
               tools="hover", tooltips="@label: @value", x_range=(-0.5, 1.0))

    config = get_config()
    config.update(user_config)

    raw_data = get_raw_data(config)
    data = gen_data(config, raw_data)
    source = ColumnDataSource(data=next(data))
    p.wedge(x=0, y=1, radius=0.4,
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
            line_color="white", fill_color='color', legend_field='label', source=source)

    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None

    t = show(p, notebook_handle=True)
    for d in data:
        source.data = d
        push_notebook(handle=t)
        time.sleep(0.02)


Loading BokehJS ...

In [5]:
plot({
    'VIRUS': {
        "R0": 2
    },
    'CYCLEs': 1000,
    'INITIAL_CASE': 100,
})

In [3]:
plot({
    'VIRUS': {
        "R0": 3
    },
    'CYCLEs': 1000,
    'INITIAL_CASE': 100,
})